In [76]:
import yfinance as yf
import numpy as np
import pandas as pd

In [ ]:
def get_financial_data(ticker):
    stock = yf.Ticker(ticker)
    cashflow = stock.cashflow
    beta = stock.info.get('beta')
    return cashflow, beta


def calculate_fcf(cashflow):
    fcf = cashflow.loc["Free Cash Flow"]  # Use Free Cash Flow directly from the cash flow statement
    fcf = fcf.dropna()
    if fcf.empty:
        return None
    return fcf


def calculate_cost_of_equity(beta, risk_free_rate=0.03, market_return=0.08):
    return risk_free_rate + beta * (market_return - risk_free_rate) if beta else None


def perform_dcf_analysis(fcf, discount_rate, terminal_growth_rate=0.02, years=3, fcf_growth_rate=0.05):
    # FCF Estimates: FCF_1 = FCF_0 * (1 + g)^n
    fcf_estimates = [fcf.iloc[0] * (1 + fcf_growth_rate) ** (i + 1) for i in range(years)]
    
    # Present Value of Future Cash Flows: PV_FCF = FCF_n / (1 + r)^n
    present_value_fcf = [fcf_estimates[i] / (1 + discount_rate) ** (i + 1) for i in range(years)]
    sum_pv_fcf = sum(present_value_fcf)
    
    # Terminal Value: TV = FCF_last * (1 + g) / (r - g)
    terminal_value = fcf_estimates[-1] * (1 + terminal_growth_rate) / (discount_rate - terminal_growth_rate)
    
    # Present Value of Terminal Value: PV_TV = TV / (1 + r)^n
    present_value_terminal = terminal_value / (1 + discount_rate) ** years
    
    return sum_pv_fcf + present_value_terminal, present_value_fcf, present_value_terminal


def format_value(value):
    if value >= 1e12:
        return f"{value / 1e12:.2f} trillion"
    elif value >= 1e9:
        return f"{value / 1e9:.2f} billion"
    elif value >= 1e6:
        return f"{value / 1e6:.2f} million"
    elif value >= 1e3:
        return f"{value / 1e3:.2f} thousand"
    else:
        return f"{value:.2f}"


def get_additional_metrics(ticker):
    stock = yf.Ticker(ticker)
    pe_ratio = stock.info.get('trailingPE', None)
    pb_ratio = stock.info.get('priceToBook', None)
    dividend_yield = stock.info.get('dividendYield', None)
    return pe_ratio, pb_ratio, dividend_yield


def main():
    tickers = [ticker.strip().upper() for ticker in input("Enter stock tickers separated by commas: ").split(",")]
    years = int(input("Enter number of years for analysis: "))
    risk_free_rate = 0.03
    market_return = 0.08
    terminal_growth_rate = 0.02

    print(f"Assumptions: Risk-Free Rate = {risk_free_rate * 100:.2f}%, Market Return = {market_return * 100:.2f}%, Terminal Growth Rate = {terminal_growth_rate * 100:.2f}%")
    print(f"Forecasting FCF for {years} years ahead.")

    results = []

    for ticker in tickers:
        print(f"\nRunning DCF analysis for {ticker}")
        cashflow, beta = get_financial_data(ticker)
        if cashflow is None or beta is None:
            print(f"Failed to get necessary financial data for {ticker}. Skipping.")
            continue

        fcf = calculate_fcf(cashflow)
        if fcf is None or fcf.empty:
            print(f"No valid Free Cash Flow data for {ticker}. Skipping.")
            continue

        discount_rate = calculate_cost_of_equity(beta, risk_free_rate=risk_free_rate, market_return=market_return)
        if discount_rate is None:
            print(f"Failed to calculate discount rate for {ticker}. Skipping.")
            continue

        # Calculate the historical FCF growth rate using a more robust method
        if len(fcf) > 1:
            fcf = fcf.infer_objects()  # Fix for future warning on downcasting
            # Use CAGR (Compound Annual Growth Rate) for a more accurate growth rate calculation
            start_value = fcf.iloc[-1]
            end_value = fcf.iloc[0]
            num_periods = len(fcf) - 1
            if start_value > 0 and end_value > 0 and num_periods > 0:
                fcf_growth_rate = ((end_value / start_value) ** (1 / num_periods)) - 1
            else:
                fcf_growth_rate = 0.05  # Default growth rate if data is insufficient or negative
            fcf_growth_rate = min(max(fcf_growth_rate, -0.05), 0.05)  # Cap growth rate between -5% and 5%
        else:
            fcf_growth_rate = 0.05

        print(f"Calculated FCF Growth Rate: {fcf_growth_rate * 100:.2f}%")
        print(f"Historical Free Cash Flow period: {fcf.index[-1].date()} to {fcf.index[0].date()}")

        total_enterprise_value, present_value_fcf, present_value_terminal = perform_dcf_analysis(
            fcf, discount_rate, terminal_growth_rate=terminal_growth_rate, years=years, fcf_growth_rate=fcf_growth_rate
        )
        current_price = yf.Ticker(ticker).history(period="1d").iloc[0]["Close"]
        shares_outstanding = yf.Ticker(ticker).info.get('sharesOutstanding')
        if shares_outstanding:
            intrinsic_value_per_share = total_enterprise_value / shares_outstanding
            valuation_message = "undervalued" if intrinsic_value_per_share > current_price else "overvalued"
        else:
            intrinsic_value_per_share = None
            valuation_message = "Could not determine valuation due to missing data"

        pe_ratio, pb_ratio, dividend_yield = get_additional_metrics(ticker)

        results.append({
            "Ticker": ticker,
            "TEV": total_enterprise_value,
            "PV_FCF": sum(present_value_fcf),
            "PV_Terminal": present_value_terminal,
            "Discount Rate": discount_rate,
            "FCF Growth Rate": fcf_growth_rate,
            "Intrinsic Value per Share": intrinsic_value_per_share,
            "Current Price": current_price,
            "Valuation": valuation_message,
            "P/E Ratio": pe_ratio,
            "P/B Ratio": pb_ratio,
            "Dividend Yield": f"{dividend_yield * 100:.2f}%" if dividend_yield else "N/A"
        })

    # Convert results to a DataFrame for easy visualization
    if results:
        df = pd.DataFrame(results)
        df['TEV'] = df['TEV'].apply(format_value)
        df['PV_FCF'] = df['PV_FCF'].apply(format_value)
        df['PV_Terminal'] = df['PV_Terminal'].apply(format_value)
        df['Discount Rate'] = df['Discount Rate'] * 100
        df['FCF Growth Rate'] = df['FCF Growth Rate'] * 100
        df['Intrinsic Value per Share'] = df['Intrinsic Value per Share'].apply(lambda x: f"${x:.2f}" if x else "N/A")
        df['Current Price'] = df['Current Price'].apply(lambda x: f"${x:.2f}")

        # Print the tabulated results
        print(df[['Ticker', 'TEV', 'PV_FCF', 'PV_Terminal', 'Discount Rate', 'FCF Growth Rate', 'Intrinsic Value per Share', 'Current Price', 'Valuation', 'P/E Ratio', 'P/B Ratio', 'Dividend Yield']].to_string(index=False))


if __name__ == "__main__":
    main()


In [89]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

# DCF Analysis Code
def get_financial_data(ticker):
    stock = yf.Ticker(ticker)
    cashflow = stock.cashflow
    beta = stock.info.get('beta')
    # Get interest expense and total debt for cost of debt calculation
    interest_expense = stock.financials.loc['Interest Expense'].fillna(0).sum() if 'Interest Expense' in stock.financials.index else None
    total_debt = stock.balance_sheet.loc['Total Debt'].fillna(0).sum() if 'Total Debt' in stock.balance_sheet.index else None
    cost_of_debt = (interest_expense / total_debt) if interest_expense is not None and total_debt else None
    return cashflow, beta, cost_of_debt


def calculate_fcf(cashflow, end_year=None):
    fcf = cashflow.loc["Free Cash Flow"]  # Use Free Cash Flow directly from the cash flow statement
    fcf = fcf.dropna()
    if end_year:
        end_year = str(end_year)
        if end_year not in fcf.index:
            print(f"Warning: No data found for year {end_year}. Defaulting to most recent period.")
            return fcf.iloc[:4]  # Default to most recent 4 years if year not found
        start_index = fcf.index.get_loc(end_year)
        if start_index + 4 > len(fcf):
            print(f"Warning: Not enough data available for a 4-year period ending {end_year}. Defaulting to most recent available data.")
            return fcf.iloc[:4]
        return fcf.iloc[start_index:start_index + 4]
    return fcf.iloc[:4]  # Default to most recent 4 years


def calculate_cost_of_equity(beta, risk_free_rate, market_return):
    return risk_free_rate + beta * (market_return - risk_free_rate) if beta else None


def calculate_wacc(beta, cost_of_debt, equity_value, debt_value, tax_rate, risk_free_rate=0.03, market_return=0.08):
    cost_of_equity = calculate_cost_of_equity(beta, risk_free_rate, market_return)
    if cost_of_equity is None or cost_of_debt is None:
        return None
    
    total_value = equity_value + debt_value
    wacc = (
        (equity_value / total_value) * cost_of_equity +
        (debt_value / total_value) * cost_of_debt * (1 - tax_rate)
    )
    return wacc


def perform_dcf_analysis(fcf, discount_rate, terminal_growth_rate=0.02, years=3, fcf_growth_rate=0.05):
    # FCF Estimates: FCF_1 = FCF_0 * (1 + g)^n
    fcf_estimates = [fcf.iloc[0] * (1 + fcf_growth_rate) ** (i + 1) for i in range(years)]
    
    # Present Value of Future Cash Flows: PV_FCF = FCF_n / (1 + r)^n
    present_value_fcf = [fcf_estimates[i] / (1 + discount_rate) ** (i + 1) for i in range(years)]
    sum_pv_fcf = sum(present_value_fcf)
    
    # Terminal Value: TV = FCF_last * (1 + g) / (r - g)
    terminal_value = fcf_estimates[-1] * (1 + terminal_growth_rate) / (discount_rate - terminal_growth_rate)
    
    # Present Value of Terminal Value: PV_TV = TV / (1 + r)^n
    present_value_terminal = terminal_value / (1 + discount_rate) ** years
    
    return sum_pv_fcf + present_value_terminal, present_value_fcf, present_value_terminal


def format_value(value):
    if value is None:
        return "N/A"
    if value >= 1e12:
        return f"{value / 1e12:.2f} trillion"
    elif value >= 1e9:
        return f"{value / 1e9:.2f} billion"
    elif value >= 1e6:
        return f"{value / 1e6:.2f} million"
    elif value >= 1e3:
        return f"{value / 1e3:.2f} thousand"
    else:
        return f"{value:.2f}"


def get_additional_metrics(ticker):
    stock = yf.Ticker(ticker)
    pe_ratio = stock.info.get('trailingPE', None)
    pb_ratio = stock.info.get('priceToBook', None)
    dividend_yield = stock.info.get('dividendYield', None)
    return pe_ratio, pb_ratio, dividend_yield


def run_dcf_analysis(tickers, years=3, end_year=None):
    risk_free_rate = 0.03
    market_return = 0.08
    terminal_growth_rate = 0.02
    tax_rate = 0.21  # Assume a corporate tax rate of 21%

    print(f"Assumptions: Risk-Free Rate = {risk_free_rate * 100:.2f}%, Market Return = {market_return * 100:.2f}%, Terminal Growth Rate = {terminal_growth_rate * 100:.2f}%")
    print(f"Forecasting FCF for {years} years ahead.")

    results = []

    for ticker in tickers:
        print(f"\nRunning DCF analysis for {ticker}")
        cashflow, beta, cost_of_debt = get_financial_data(ticker)
        if cashflow is None or beta is None:
            print(f"Failed to get necessary financial data for {ticker}. Skipping.")
            continue

        fcf = calculate_fcf(cashflow, end_year)
        if fcf is None or fcf.empty:
            print(f"No valid Free Cash Flow data for {ticker}. Skipping.")
            continue

        if fcf.isna().any():
            print(f"Warning: Free Cash Flow for {ticker} contains NaN values. Adjusting calculation.")
            fcf = fcf.dropna()

        discount_rate = calculate_cost_of_equity(beta, risk_free_rate=risk_free_rate, market_return=market_return)
        equity_value = yf.Ticker(ticker).info.get('marketCap', 0)
        debt_value = yf.Ticker(ticker).info.get('totalDebt', 0)

        wacc = calculate_wacc(beta, cost_of_debt, equity_value, debt_value, tax_rate, risk_free_rate, market_return)
        if wacc is None:
            print(f"Failed to calculate WACC for {ticker}. Using cost of equity as discount rate.")
            wacc = discount_rate

        # Calculate the historical FCF growth rate using a more robust method
        if len(fcf) > 1:
            fcf = fcf.infer_objects()  # Fix for future warning on downcasting
            # Use CAGR (Compound Annual Growth Rate) for a more accurate growth rate calculation
            start_value = fcf.iloc[-1]
            end_value = fcf.iloc[0]
            num_periods = len(fcf) - 1
            if start_value > 0 and end_value > 0 and num_periods > 0:
                fcf_growth_rate = ((end_value / start_value) ** (1 / num_periods)) - 1
            else:
                fcf_growth_rate = 0.05  # Default growth rate if data is insufficient or negative
            fcf_growth_rate = min(max(fcf_growth_rate, -0.05), 0.05)  # Cap growth rate between -5% and 5%
        else:
            fcf_growth_rate = 0.05

        print(f"Calculated FCF Growth Rate: {fcf_growth_rate * 100:.2f}%")
        print(f"Historical Free Cash Flow period: {fcf.index[-1].date()} to {fcf.index[0].date()}")

        total_enterprise_value, present_value_fcf, present_value_terminal = perform_dcf_analysis(
            fcf, wacc, terminal_growth_rate=terminal_growth_rate, years=years, fcf_growth_rate=fcf_growth_rate
        )
        current_price = yf.Ticker(ticker).history(period="1d").iloc[0]["Close"]
        shares_outstanding = yf.Ticker(ticker).info.get('sharesOutstanding')
        if shares_outstanding:
            intrinsic_value_per_share = total_enterprise_value / shares_outstanding
            valuation_message = "undervalued" if intrinsic_value_per_share > current_price else "overvalued"
        else:
            intrinsic_value_per_share = None
            valuation_message = "Could not determine valuation due to missing data"

        pe_ratio, pb_ratio, dividend_yield = get_additional_metrics(ticker)

        results.append({
            "Ticker": ticker,
            "TEV": total_enterprise_value,
            "PV_FCF": sum(present_value_fcf),
            "PV_Terminal": present_value_terminal,
            "Discount Rate": wacc,
            "FCF Growth Rate": fcf_growth_rate,
            "Intrinsic Value per Share": intrinsic_value_per_share,
            "Current Price": current_price,
            "Valuation": valuation_message,
            "P/E Ratio": pe_ratio,
            "P/B Ratio": pb_ratio,
            "Dividend Yield": f"{dividend_yield * 100:.2f}%" if dividend_yield else "N/A"
        })

    # Convert results to a DataFrame for easy visualization
    if results:
        df = pd.DataFrame(results)
        df['TEV'] = df['TEV'].apply(format_value)
        df['PV_FCF'] = df['PV_FCF'].apply(format_value)
        df['PV_Terminal'] = df['PV_Terminal'].apply(format_value)
        df['Discount Rate'] = df['Discount Rate'] * 100
        df['Discount Rate'] = df['Discount Rate'].apply(lambda x: f"{x:.2f}%")
        df['Intrinsic Value per Share'] = df['Intrinsic Value per Share'].apply(format_value)
        df['Current Price'] = df['Current Price'].apply(format_value)
        print(df)

# Comparable Company Analysis (CCA)
def run_comparable_analysis(tickers):
    comp_data = []
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        pe_ratio, pb_ratio, dividend_yield = get_additional_metrics(ticker)
        ev_to_ebitda = stock.info.get('enterpriseToEbitda', None)
        comp_data.append({
            "Ticker": ticker,
            "P/E Ratio": pe_ratio,
            "P/B Ratio": pb_ratio,
            "EV/EBITDA Ratio": ev_to_ebitda,
            "Dividend Yield": f"{dividend_yield * 100:.2f}%" if dividend_yield else "N/A"
        })

    # Convert results to a DataFrame for easy visualization
    df = pd.DataFrame(comp_data)
    for col in ['P/E Ratio', 'P/B Ratio', 'EV/EBITDA Ratio']:
        df[col] = df[col].apply(format_value)
    print("\nComparable Company Analysis:")
    print(df)

if __name__ == "__main__":
    tickers = input("Enter stock tickers separated by commas (e.g., 'TSM, AAPL, MSFT'): ").split(',')
    tickers = [ticker.strip().upper() for ticker in tickers]
    years = input("Enter number of years for DCF analysis (default is 3): ")
    years = int(years) if years else 3
    end_year = input("Enter end year for historical FCF period (e.g., '2023') or press Enter to use most recent: ")
    end_year = int(end_year) if end_year else None

    run_dcf_analysis(tickers, years=years, end_year=end_year)
    print("\n--- Running Comparable Company Analysis ---")
    run_comparable_analysis(tickers)


Assumptions: Risk-Free Rate = 3.00%, Market Return = 8.00%, Terminal Growth Rate = 2.00%
Forecasting FCF for 3 years ahead.

Running DCF analysis for TSM


/var/folders/cz/b459nk9s47n9_9ykqvrh5tzh0000gn/T/ipykernel_86859/3192218682.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  interest_expense = stock.financials.loc['Interest Expense'].fillna(0).sum() if 'Interest Expense' in stock.financials.index else None
/var/folders/cz/b459nk9s47n9_9ykqvrh5tzh0000gn/T/ipykernel_86859/3192218682.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  total_debt = stock.balance_sheet.loc['Total Debt'].fillna(0).sum() if 'Total Debt' in stock.balance_sheet.index else None


Calculated FCF Growth Rate: -2.15%
Historical Free Cash Flow period: 2020-12-31 to 2023-12-31

Running DCF analysis for AAPL


/var/folders/cz/b459nk9s47n9_9ykqvrh5tzh0000gn/T/ipykernel_86859/3192218682.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  interest_expense = stock.financials.loc['Interest Expense'].fillna(0).sum() if 'Interest Expense' in stock.financials.index else None
/var/folders/cz/b459nk9s47n9_9ykqvrh5tzh0000gn/T/ipykernel_86859/3192218682.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  total_debt = stock.balance_sheet.loc['Total Debt'].fillna(0).sum() if 'Total Debt' in stock.balance_sheet.index else None


Calculated FCF Growth Rate: 5.00%
Historical Free Cash Flow period: 2020-09-30 to 2023-09-30

Running DCF analysis for MSFT


/var/folders/cz/b459nk9s47n9_9ykqvrh5tzh0000gn/T/ipykernel_86859/3192218682.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  interest_expense = stock.financials.loc['Interest Expense'].fillna(0).sum() if 'Interest Expense' in stock.financials.index else None
/var/folders/cz/b459nk9s47n9_9ykqvrh5tzh0000gn/T/ipykernel_86859/3192218682.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  total_debt = stock.balance_sheet.loc['Total Debt'].fillna(0).sum() if 'Total Debt' in stock.balance_sheet.index else None


Calculated FCF Growth Rate: 5.00%
Historical Free Cash Flow period: 2021-06-30 to 2024-06-30
  Ticker            TEV          PV_FCF    PV_Terminal Discount Rate  \
0    TSM  8.06 trillion  744.84 billion  7.31 trillion         5.22%   
1   AAPL  1.58 trillion  277.40 billion  1.30 trillion         8.99%   
2   MSFT  1.54 trillion  212.67 billion  1.32 trillion         7.34%   

   FCF Growth Rate Intrinsic Value per Share Current Price    Valuation  \
0        -0.021509             1.55 thousand        210.75  undervalued   
1         0.050000                    103.68        232.18   overvalued   
2         0.050000                    206.81        417.40   overvalued   

   P/E Ratio  P/B Ratio Dividend Yield  
0  37.566845   1.441292          1.32%  
1  35.393290  52.984936          0.43%  
2  35.313454  11.557662          0.80%  

--- Running Comparable Company Analysis ---

Comparable Company Analysis:
  Ticker P/E Ratio P/B Ratio EV/EBITDA Ratio Dividend Yield
0    TSM     37.57

In [94]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

# DCF Analysis Code
def get_financial_data(ticker):
    stock = yf.Ticker(ticker)
    cashflow = stock.cashflow
    beta = stock.info.get('beta')
    # Get interest expense and total debt for cost of debt calculation
    interest_expense = stock.financials.loc['Interest Expense'].infer_objects(copy=False).sum() if 'Interest Expense' in stock.financials.index else None
    total_debt = stock.balance_sheet.loc['Total Debt'].infer_objects(copy=False).sum() if 'Total Debt' in stock.balance_sheet.index else None
    cost_of_debt = (interest_expense / total_debt) if interest_expense is not None and total_debt else None
    return cashflow, beta, cost_of_debt


def calculate_fcf(cashflow, end_year=None):
    fcf = cashflow.loc["Free Cash Flow"]  # Use Free Cash Flow directly from the cash flow statement
    fcf = fcf.dropna()
    if end_year:
        end_year = str(end_year)
        if end_year not in fcf.index:
            print(f"Warning: No data found for year {end_year}. Defaulting to most recent period.")
            return fcf.iloc[:4]  # Default to most recent 4 years if year not found
        start_index = fcf.index.get_loc(end_year)
        if start_index + 4 > len(fcf):
            print(f"Warning: Not enough data available for a 4-year period ending {end_year}. Defaulting to most recent available data.")
            return fcf.iloc[:4]
        return fcf.iloc[start_index:start_index + 4]
    return fcf.iloc[:4]  # Default to most recent 4 years


def calculate_cost_of_equity(beta, risk_free_rate, market_return):
    return risk_free_rate + beta * (market_return - risk_free_rate) if beta else None


def calculate_wacc(beta, cost_of_debt, equity_value, debt_value, tax_rate, risk_free_rate=0.03, market_return=0.08):
    cost_of_equity = calculate_cost_of_equity(beta, risk_free_rate, market_return)
    if cost_of_equity is None or cost_of_debt is None:
        return None
    
    total_value = equity_value + debt_value
    wacc = (
        (equity_value / total_value) * cost_of_equity +
        (debt_value / total_value) * cost_of_debt * (1 - tax_rate)
    )
    return wacc


def perform_dcf_analysis(fcf, discount_rate, terminal_growth_rate=0.02, years=3, fcf_growth_rate=0.05):
    # FCF Estimates: FCF_1 = FCF_0 * (1 + g)^n
    fcf_estimates = [fcf.iloc[0] * (1 + fcf_growth_rate) ** (i + 1) for i in range(years)]
    
    # Present Value of Future Cash Flows: PV_FCF = FCF_n / (1 + r)^n
    present_value_fcf = [fcf_estimates[i] / (1 + discount_rate) ** (i + 1) for i in range(years)]
    sum_pv_fcf = sum(present_value_fcf)
    
    # Terminal Value: TV = FCF_last * (1 + g) / (r - g)
    terminal_value = fcf_estimates[-1] * (1 + terminal_growth_rate) / (discount_rate - terminal_growth_rate)
    
    # Present Value of Terminal Value: PV_TV = TV / (1 + r)^n
    present_value_terminal = terminal_value / (1 + discount_rate) ** years
    
    return sum_pv_fcf + present_value_terminal, present_value_fcf, present_value_terminal


def format_value(value):
    if value is None:
        return "N/A"
    if value >= 1e12:
        return f"{value / 1e12:.2f} trillion"
    elif value >= 1e9:
        return f"{value / 1e9:.2f} billion"
    elif value >= 1e6:
        return f"{value / 1e6:.2f} million"
    elif value >= 1e3:
        return f"{value / 1e3:.2f} thousand"
    else:
        return f"{value:.2f}"


def get_additional_metrics(ticker):
    stock = yf.Ticker(ticker)
    pe_ratio = stock.info.get('trailingPE', None)
    pb_ratio = stock.info.get('priceToBook', None)
    dividend_yield = stock.info.get('dividendYield', None)
    return pe_ratio, pb_ratio, dividend_yield


def run_dcf_analysis(tickers, years=3, end_year=None):
    risk_free_rate = 0.03
    market_return = 0.08
    terminal_growth_rate = 0.02
    tax_rate = 0.21  # Assume a corporate tax rate of 21%

    print(f"Assumptions: Risk-Free Rate = {risk_free_rate * 100:.2f}%, Market Return = {market_return * 100:.2f}%, Terminal Growth Rate = {terminal_growth_rate * 100:.2f}%")
    print(f"Forecasting FCF for {years} years ahead.")

    results = []

    for ticker in tickers:
        print(f"\nRunning DCF analysis for {ticker}")
        cashflow, beta, cost_of_debt = get_financial_data(ticker)
        if cashflow is None or beta is None:
            print(f"Failed to get necessary financial data for {ticker}. Skipping.")
            continue

        fcf = calculate_fcf(cashflow, end_year)
        if fcf is None or fcf.empty:
            print(f"No valid Free Cash Flow data for {ticker}. Skipping.")
            continue

        if fcf.isna().any():
            print(f"Warning: Free Cash Flow for {ticker} contains NaN values. Adjusting calculation.")
            fcf = fcf.dropna()

        discount_rate = calculate_cost_of_equity(beta, risk_free_rate=risk_free_rate, market_return=market_return)
        equity_value = yf.Ticker(ticker).info.get('marketCap', 0)
        debt_value = yf.Ticker(ticker).info.get('totalDebt', 0)

        wacc = calculate_wacc(beta, cost_of_debt, equity_value, debt_value, tax_rate, risk_free_rate, market_return)
        if wacc is None:
            print(f"Failed to calculate WACC for {ticker}. Using cost of equity as discount rate.")
            wacc = discount_rate

        # Calculate the historical FCF growth rate using a more robust method
        if len(fcf) > 1:
            fcf = fcf.infer_objects()  # Fix for future warning on downcasting
            # Use CAGR (Compound Annual Growth Rate) for a more accurate growth rate calculation
            start_value = fcf.iloc[-1]
            end_value = fcf.iloc[0]
            num_periods = len(fcf) - 1
            if start_value > 0 and end_value > 0 and num_periods > 0:
                fcf_growth_rate = ((end_value / start_value) ** (1 / num_periods)) - 1
            else:
                fcf_growth_rate = 0.05  # Default growth rate if data is insufficient or negative
            fcf_growth_rate = min(max(fcf_growth_rate, -0.05), 0.05)  # Cap growth rate between -5% and 5%
        else:
            fcf_growth_rate = 0.05

        print(f"Calculated FCF Growth Rate: {fcf_growth_rate * 100:.2f}%")
        print(f"Historical Free Cash Flow period: {fcf.index[-1].date()} to {fcf.index[0].date()}")

        total_enterprise_value, present_value_fcf, present_value_terminal = perform_dcf_analysis(
            fcf, wacc, terminal_growth_rate=terminal_growth_rate, years=years, fcf_growth_rate=fcf_growth_rate
        )
        current_price = yf.Ticker(ticker).history(period="1d").iloc[0]["Close"]
        shares_outstanding = yf.Ticker(ticker).info.get('sharesOutstanding')
        if shares_outstanding:
            intrinsic_value_per_share = total_enterprise_value / shares_outstanding
            valuation_message = "undervalued" if intrinsic_value_per_share > current_price else "overvalued"
        else:
            intrinsic_value_per_share = None
            valuation_message = "Could not determine valuation due to missing data"

        pe_ratio, pb_ratio, dividend_yield = get_additional_metrics(ticker)

        results.append({
            "Ticker": ticker,
            "TEV": total_enterprise_value,
            "PV_FCF": sum(present_value_fcf),
            "PV_Terminal": present_value_terminal,
            "Discount Rate": wacc,
            "FCF Growth Rate": fcf_growth_rate,
            "Intrinsic Value per Share": intrinsic_value_per_share,
            "Current Price": current_price,
            "Valuation": valuation_message,
            "P/E Ratio": pe_ratio,
            "P/B Ratio": pb_ratio,
            "Dividend Yield": dividend_yield
        })

    results_df = pd.DataFrame(results)
    results_df['TEV'] = results_df['TEV'].apply(format_value)
    results_df['PV_FCF'] = results_df['PV_FCF'].apply(format_value)
    results_df['PV_Terminal'] = results_df['PV_Terminal'].apply(format_value)
    results_df['Intrinsic Value per Share'] = results_df['Intrinsic Value per Share'].apply(lambda x: format_value(x) if x else 'N/A')
    results_df['Discount Rate'] = results_df['Discount Rate'].apply(lambda x: f"{x * 100:.2f}%" if x else 'N/A')
    results_df['FCF Growth Rate'] = results_df['FCF Growth Rate'].apply(lambda x: f"{x * 100:.2f}%")
    results_df['Dividend Yield'] = results_df['Dividend Yield'].apply(lambda x: f"{x * 100:.2f}%" if x else 'N/A')

    print("\n--- DCF Analysis Results ---")
    print(results_df)


if __name__ == "__main__":
    tickers = input("Enter stock tickers separated by commas (e.g., 'AAPL, MSFT'): ").split(',')
    tickers = [ticker.strip().upper() for ticker in tickers]
    years = input("Enter number of years for DCF analysis (default is 3): ")
    years = int(years) if years else 3
    end_year = input("Enter end year for historical FCF period (default is most recent): ")
    end_year = int(end_year) if end_year else None

    run_dcf_analysis(tickers, years=years, end_year=end_year)
    print("\n--- Running Comparable Company Analysis ---")
    run_comparable_analysis(tickers)


Assumptions: Risk-Free Rate = 3.00%, Market Return = 8.00%, Terminal Growth Rate = 2.00%
Forecasting FCF for 3 years ahead.

Running DCF analysis for AAPL
Calculated FCF Growth Rate: 5.00%
Historical Free Cash Flow period: 2020-09-30 to 2023-09-30

Running DCF analysis for TSM
Calculated FCF Growth Rate: -2.15%
Historical Free Cash Flow period: 2020-12-31 to 2023-12-31

Running DCF analysis for MSFT
Calculated FCF Growth Rate: 5.00%
Historical Free Cash Flow period: 2021-06-30 to 2024-06-30

--- DCF Analysis Results ---
  Ticker            TEV          PV_FCF    PV_Terminal Discount Rate  \
0   AAPL  1.58 trillion  277.40 billion  1.30 trillion         8.99%   
1    TSM  8.08 trillion  744.96 billion  7.33 trillion         5.21%   
2   MSFT  1.54 trillion  212.67 billion  1.32 trillion         7.34%   

  FCF Growth Rate Intrinsic Value per Share  Current Price    Valuation  \
0           5.00%                    103.68     232.259995   overvalued   
1          -2.15%             1.56 

In [93]:
get_financial_data("TSM")
stock.financials

,2023-12-31,2022-12-31,2021-12-31,2020-12-31,2019-12-31
Tax Effect Of Unusual Items,439276610.819893,450946515.219931,612244570.460729,804033135.053612,NaN
Tax Rate For Calcs,0.130998,0.13179,0.105809,0.126103,NaN
Normalized EBITDA,1520153500000.0,1589654800000.0,1085058800000.0,912176400000.0,NaN
Total Unusual Items,3353300000.0,3421700000.0,5786300000.0,6376000000.0,NaN
Total Unusual Items Excluding Goodwill,3353300000.0,3421700000.0,5786300000.0,6376000000.0,NaN
Net Income From Continuing Operation Net Minority Interest,851740000000.0,992923400000.0,592359200000.0,510744000000.0,NaN
Reconciled Depreciation,532190900000.0,437254300000.0,422394900000.0,331724600000.0,NaN
Reconciled Cost Of Revenue,986625200000.0,915536500000.0,767877700000.0,628124700000.0,NaN
EBITDA,1523506800000.0,1593076500000.0,1090845100000.0,918552400000.0,NaN
EBIT,991315900000.0,1155822200000.0,668450200000.0,586827800000.0,NaN
